In [9]:
import pandas as pd

In [10]:
data = pd.read_csv('isp_data.csv').iloc[:,1:]

In [22]:
import pandas as pd
import requests
import random

base_url = "http://0.0.0.0:8000"

# Identify unique customers
unique_customers = data['account_id'].unique()


# Dictionary to store the mapping between account_id and customer_id
account_to_customer_id = {}

for account_id in unique_customers:
    # Generate age and gender
    age = random.randint(20, 70)
    gender = random.choice(["Male", "Female"])

    # Insert customer into Dim_Customer
    customer_data = {
        "age": age,
        "gender": gender
    }
    response = requests.post(f"{base_url}/customer/", json=customer_data)
    if response.status_code != 200:
        print(f"Failed to insert customer {account_id}. Response: {response.text}")
        continue

    customer_id = response.json()['customer_id']
    
    # Store the mapping
    account_to_customer_id[account_id] = customer_id

# Add customer_id to the dataset
data['customer_id'] = data['account_id'].map(account_to_customer_id)


In [25]:
# Identify unique payment methods
unique_payment_methods = data['payment_method'].unique()

# Dictionary to store the mapping between payment_method and payment_method_id
payment_to_payment_id = {}

for payment in unique_payment_methods:
    # Insert payment method into Dim_Payment_Method
    payment_data = {
        "payment_method_name": payment
    }
    response = requests.post(f"{base_url}/payment_method/", json=payment_data)
    if response.status_code != 200:
        print(f"Failed to insert payment method {payment}. Response: {response.text}")
        continue

    payment_method_id = response.json()['payment_method_id']
    
    # Store the mapping
    payment_to_payment_id[payment] = payment_method_id

# Add payment_method_id to the dataset
data['payment_method_id'] = data['payment_method'].map(payment_to_payment_id)


In [28]:
# Insert each row into the Fact_Transacation table
for index, row in data.iterrows():
    transaction_data = {
        "customer_id": row['customer_id'],
        "payment_method_id": row['payment_method_id'],
        "transaction_date": row['payment_date'],
        "amount": row['payment_amount']
    }
    
    response = requests.post(f"{base_url}/transaction/", json=transaction_data)
    if response.status_code != 200:
        print(f"Failed to insert transaction for row {index}. Response: {response.text}")